In [1]:
import pandas as pd
import numpy as np
import viet_lib as vi
from gensim import corpora, models, similarities
import jieba
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
##LOAD EMOJICON
file = open('files/emojicon.txt', 'r', encoding="utf8")
emoji_lst = file.read().split('\n')
emoji_dict = {}
for line in emoji_lst:
    key, value = line.split('\t')
    emoji_dict[key] = str(value)
file.close()
#################
#LOAD TEENCODE
file = open('files/teencode.txt', 'r', encoding="utf8")
teen_lst = file.read().split('\n')
teen_dict = {}
for line in teen_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
file.close()
###############
#LOAD TRANSLATE ENGLISH -> VNMESE
file = open('files/english-vnmese.txt', 'r', encoding="utf8")
english_lst = file.read().split('\n')
english_dict = {}
for line in english_lst:
    key, value = line.split('\t')
    english_dict[key] = str(value)
file.close()
################
#LOAD wrong words
file = open('files/wrong-word.txt', 'r', encoding="utf8")
wrong_lst = file.read().split('\n')
file.close()
#################
#LOAD STOPWORDS
file = open('files/vietnamese-stopwords.txt', 'r', encoding="utf8")
stopwords_lst = file.read().split('\n')
file.close()

In [3]:
def handle_text(text, emoji_dict, teen_dict, wrong_lst, stopwords_lst):
    document = vi.process_text(text, emoji_dict, teen_dict, wrong_lst)
    document = vi.covert_unicode(document)
    document = vi.process_postag_thesea(document)
    document = vi.remove_stopword(document, stopwords_lst)
    return document

def count_text(text):
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(text)
    return vectorizer.get_feature_names()

def get_index_top_5(sim):
    return np.delete(sim.argsort()[-6:][::-1], [0])

In [4]:
df = pd.read_csv('data.csv')

In [5]:
df['content'] = df.apply(lambda x: handle_text(x['lyric'], emoji_dict, teen_dict, wrong_lst, stopwords_lst), axis=1)
df.head()

,name,singer,lyric,content
0,Nhạc Tình Đêm Mưa,Hà Thanh Xuân,Mưa vẫn rơi rơi. Mưa giữa đêm khuya. Như tiếng...,rơi rơi mưa buông khóc rơi rơi vương buồn tiếc...
1,Như Hoa Mùa Xuân,"Hồ Ngọc Hà, Thủy Tiên, Minh Hằng",Xuân mang niềm vui đến. Xuân mang tình yêu tới...,tình_yêu đào tràn rộn_ràng bình_yên đón ngân c...
2,Giai Điệu Mùa Xuân,Noo Phước Thịnh,Cành mai tươi thắm đón xuân về đây. Lộc xuân c...,tươi_thắm đón đùa náo_nức tiếng_nói trẻ_thơ há...
3,Mùa Xuân Ơi Mùa Xuân,Bé Thảo Nguyên,Hãy đến mùa xuân ơi. Cho nụ cười càng thêm tro...,cười thoáng gọi mừng êm_ái trôi cất_tiếng cười...
4,"Mùa Xuân, Mùa Xuân",Mỹ Tâm,Hãy đến mùa xuân ơi. Cho nụ cười càng thêm tro...,cười gợi trở mừng nhẹ nhẹ trôi cười cười mãi n...


In [6]:
df['count_text'] = df.apply(lambda x: count_text([x['content']]), axis=1)
df.head()

,name,singer,lyric,content,count_text
0,Nhạc Tình Đêm Mưa,Hà Thanh Xuân,Mưa vẫn rơi rơi. Mưa giữa đêm khuya. Như tiếng...,rơi rơi mưa buông khóc rơi rơi vương buồn tiếc...,"[buông, buồn, cũ, khóc, khẽ, mê, mưa, mộng, ná..."
1,Như Hoa Mùa Xuân,"Hồ Ngọc Hà, Thủy Tiên, Minh Hằng",Xuân mang niềm vui đến. Xuân mang tình yêu tới...,tình_yêu đào tràn rộn_ràng bình_yên đón ngân c...,"[bình_yên, chào, chúc, căng, cười, dâng, hát, ..."
2,Giai Điệu Mùa Xuân,Noo Phước Thịnh,Cành mai tươi thắm đón xuân về đây. Lộc xuân c...,tươi_thắm đón đùa náo_nức tiếng_nói trẻ_thơ há...,"[chào, chúc, cầm, hát, hòa_khắp, khắp, mắn, ná..."
3,Mùa Xuân Ơi Mùa Xuân,Bé Thảo Nguyên,Hãy đến mùa xuân ơi. Cho nụ cười càng thêm tro...,cười thoáng gọi mừng êm_ái trôi cất_tiếng cười...,"[cười, cất_tiếng, gọi, hát, mãi, mừng, nở, tho..."
4,"Mùa Xuân, Mùa Xuân",Mỹ Tâm,Hãy đến mùa xuân ơi. Cho nụ cười càng thêm tro...,cười gợi trở mừng nhẹ nhẹ trôi cười cười mãi n...,"[cười, gợi, hát, mãi, mừng, nhẹ, nở, thơm, trô..."


In [7]:
docs = df['count_text'].values.tolist()
docs[-5:]

[['chẵng',
  'chờ',
  'hạnh_phúc',
  'kề',
  'lãng_quên',
  'nỡ',
  'quên',
  'thầm',
  'vẽ',
  'yêu',
  'ôm',
  'đau',
  'đi',
  'đông',
  'ấm'],
 ['chờ',
  'cầm',
  'dũng',
  'gác',
  'hút',
  'khuất',
  'lạnh',
  'ngóng',
  'nắng',
  'thành',
  'thân',
  'trào',
  'đan',
  'đem',
  'đào',
  'đông',
  'đứng'],
 ['buốt',
  'chẳng',
  'giấu',
  'hoài',
  'khô',
  'muôn_trùng',
  'mãi_mãi',
  'sâu',
  'trôi',
  'trở',
  'yêu',
  'đau',
  'đón',
  'đông',
  'đợi_chờ'],
 ['buồn',
  'gót',
  'hẹn',
  'khô',
  'lạnh',
  'lẽ',
  'mãi',
  'mòn',
  'ngỡ',
  'quên',
  'rã',
  'rời',
  'trông',
  'xăm',
  'yêu',
  'đau'],
 ['chìm',
  'cười',
  'cầm',
  'hát',
  'lạnh',
  'mãi',
  'rơi',
  'rớt',
  'sánh',
  'trôi',
  'trở',
  'vắng',
  'xót',
  'xưa',
  'yêu',
  'đau',
  'đi',
  'đông',
  'ước']]

In [8]:
dic = corpora.Dictionary(docs)
dic.token2id

{'buông': 0,
 'buồn': 1,
 'cũ': 2,
 'khóc': 3,
 'khẽ': 4,
 'mê': 5,
 'mưa': 6,
 'mộng': 7,
 'nát': 8,
 'rơi': 9,
 'rớt': 10,
 'tiếc_thương': 11,
 'vương': 12,
 'vỡ': 13,
 'đi': 14,
 'bình_yên': 15,
 'chào': 16,
 'chúc': 17,
 'căng': 18,
 'cười': 19,
 'dâng': 20,
 'hát': 21,
 'hân': 22,
 'lấp_lánh': 23,
 'mừng': 24,
 'ngân': 25,
 'nồng': 26,
 'rộn_ràng': 27,
 'tràn': 28,
 'trái': 29,
 'tình_yêu': 30,
 'tươi': 31,
 'yêu': 32,
 'yêu_người': 33,
 'yêu_đời': 34,
 'đào': 35,
 'đón': 36,
 'ấm': 37,
 'cầm': 38,
 'hòa_khắp': 39,
 'khắp': 40,
 'mắn': 41,
 'náo_nức': 42,
 'nở': 43,
 'tiếng_nói': 44,
 'trẻ_thơ': 45,
 'tươi_thắm': 46,
 'yên': 47,
 'đua': 48,
 'đùa': 49,
 'cất_tiếng': 50,
 'gọi': 51,
 'mãi': 52,
 'thoáng': 53,
 'thơm': 54,
 'trôi': 55,
 'trọn': 56,
 'tựa': 57,
 'êm_ái': 58,
 'gợi': 59,
 'nhẹ': 60,
 'trông': 61,
 'trộn': 62,
 'trở': 63,
 'gấm': 64,
 'kính_chúc': 65,
 'tươi_cười': 66,
 'vàng': 67,
 'vẫy': 68,
 'buôn': 69,
 'bầu_trời': 70,
 'chờ': 71,
 'lân': 72,
 'múa': 73,
 'ngập': 7

In [9]:
doc2bow = [dic.doc2bow(text) for text in docs]
doc2bow[-5:]

[[(14, 1),
  (32, 1),
  (37, 1),
  (71, 1),
  (125, 1),
  (138, 1),
  (139, 1),
  (175, 1),
  (208, 1),
  (282, 1),
  (370, 1),
  (424, 1),
  (611, 1),
  (669, 1),
  (755, 1)],
 [(35, 1),
  (38, 1),
  (71, 1),
  (102, 1),
  (160, 1),
  (164, 1),
  (169, 1),
  (170, 1),
  (208, 1),
  (265, 1),
  (275, 1),
  (386, 1),
  (392, 1),
  (548, 1),
  (673, 1),
  (756, 1),
  (757, 1)],
 [(32, 1),
  (36, 1),
  (55, 1),
  (63, 1),
  (99, 1),
  (174, 1),
  (185, 1),
  (195, 1),
  (208, 1),
  (248, 1),
  (370, 1),
  (378, 1),
  (427, 1),
  (576, 1),
  (604, 1)],
 [(1, 1),
  (32, 1),
  (52, 1),
  (61, 1),
  (102, 1),
  (138, 1),
  (149, 1),
  (221, 1),
  (248, 1),
  (370, 1),
  (401, 1),
  (423, 1),
  (585, 1),
  (758, 1),
  (759, 1),
  (760, 1)],
 [(9, 1),
  (10, 1),
  (14, 1),
  (19, 1),
  (21, 1),
  (32, 1),
  (38, 1),
  (52, 1),
  (55, 1),
  (63, 1),
  (77, 1),
  (102, 1),
  (180, 1),
  (208, 1),
  (211, 1),
  (258, 1),
  (268, 1),
  (370, 1),
  (719, 1)]]

In [10]:
tfidf = models.TfidfModel(doc2bow)
index = similarities.SparseMatrixSimilarity(tfidf[doc2bow], num_features = len(dic.token2id))

In [12]:
df_result = pd.DataFrame(columns=['song_id', 'similarities_id'])
for i in range(len(df)):
    search_song = df.iloc[i]
    kw_vector = dic.doc2bow(search_song['count_text'])
    sim = index[tfidf[kw_vector]]
    top_5_sim = get_index_top_5(sim)
    if i == 0:
        print(top_5_sim)
        print(type(top_5_sim))
    df_result = df_result.append({'song_id': i, 'similarities_id': top_5_sim}, ignore_index=True)

[ 66 114  58  52  97]
<class 'numpy.ndarray'>


In [13]:
df_result.head()

,song_id,similarities_id
0,0,"[66, 114, 58, 52, 97]"
1,1,"[28, 16, 45, 7, 25]"
2,2,"[16, 28, 40, 22, 1]"
3,3,"[4, 26, 56, 132, 49]"
4,4,"[3, 136, 26, 24, 37]"


In [14]:
len(df_result)

155

In [15]:
df_result.to_csv('result.csv', index=False)

In [16]:
df = pd.read_csv('result.csv')
df.head()

,song_id,similarities_id
0,0,[ 66 114 58 52 97]
1,1,[28 16 45 7 25]
2,2,[16 28 40 22 1]
3,3,[ 4 26 56 132 49]
4,4,[ 3 136 26 24 37]


In [ ]:
df_result.info()

In [ ]:
df.info()